In [1]:
import os
import numpy as np
from tensorflow.keras.applications import VGG16, InceptionV3, ResNet50, MobileNetV2, InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.utils import to_categorical

# Dataset paths
dataset_dir = "/kaggle/input/aug-v100"
train_folder = os.path.join(dataset_dir, "output")
val_folder = os.path.join(dataset_dir, "val")
test_folder = os.path.join(dataset_dir, "test")
folders = ["Diabetic Retinopathy", "Glaucoma", "Healthy", "Macular Scar", "Myopia"]
image_size = 224  # Input image size for ResNet50
batch_size = 32
num_classes = len(folders)

# Data generators
datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = datagen.flow_from_directory(train_folder, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical')
val_generator = datagen.flow_from_directory(val_folder, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical')
test_generator = datagen.flow_from_directory(test_folder, target_size=(image_size, image_size), batch_size=1, class_mode='categorical', shuffle=False)

# Define function to create and train models
def create_model(base_model, input_size, num_classes):
    base = base_model(weights="imagenet", include_top=False, input_shape=(input_size, input_size, 3))
    x = GlobalAveragePooling2D()(base.output)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base.input, outputs=predictions)
    return model

def train_model(model, train_gen, val_gen, epochs=10):
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_gen, validation_data=val_gen, epochs=epochs, verbose=1)
    return model

# Train individual models
models = {
    "VGG16": VGG16,
    "InceptionV3": InceptionV3,
    "ResNet50": ResNet50,
    "MobileNetV2": MobileNetV2,
    "InceptionResNetV2": InceptionResNetV2
}
trained_models = {}

for model_name, model_fn in models.items():
    print(f"Training {model_name}...")
    model = create_model(model_fn, image_size, num_classes)
    trained_model = train_model(model, train_generator, val_generator, epochs=10)
    trained_models[model_name] = trained_model

# Evaluate individual models
def evaluate_model(model, test_gen):
    predictions = model.predict(test_gen, verbose=1)
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_gen.classes
    
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    report = classification_report(y_true, y_pred, target_names=folders)
    
    return acc, precision, recall, f1, report

for model_name, model in trained_models.items():
    print(f"Evaluating {model_name}...")
    acc, precision, recall, f1, report = evaluate_model(model, test_generator)
    print(f"Accuracy: {acc}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(report)

# Ensemble predictions
ensemble_predictions = np.zeros((test_generator.samples, num_classes))

for model_name, model in trained_models.items():
    predictions = model.predict(test_generator, verbose=1)
    ensemble_predictions += predictions

ensemble_predictions /= len(trained_models)
y_pred_ensemble = np.argmax(ensemble_predictions, axis=1)
y_true = test_generator.classes

ensemble_acc = accuracy_score(y_true, y_pred_ensemble)
ensemble_precision = precision_score(y_true, y_pred_ensemble, average='weighted')
ensemble_recall = recall_score(y_true, y_pred_ensemble, average='weighted')
ensemble_f1 = f1_score(y_true, y_pred_ensemble, average='weighted')
ensemble_report = classification_report(y_true, y_pred_ensemble, target_names=folders)

print("Ensemble Model Evaluation:")
print(f"Accuracy: {ensemble_acc}")
print(f"Precision: {ensemble_precision}")
print(f"Recall: {ensemble_recall}")
print(f"F1 Score: {ensemble_f1}")
print(ensemble_report)


Found 5131 images belonging to 5 classes.
Found 484 images belonging to 5 classes.
Found 482 images belonging to 5 classes.
Training VGG16...
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 78s 347ms/step - accuracy: 0.3383 - loss: 1.4450 - val_accuracy: 0.5826 - val_loss: 0.9524
Epoch 2/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 38s 231ms/step - accuracy: 0.5662 - loss: 1.0154 - val_accuracy: 0.6302 - val_loss: 0.8836
Epoch 3/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 38s 232ms/step - accuracy: 0.6852 - loss: 0.7687 - val_accuracy: 0.6591 - val_loss: 0.8363
Epoch 4/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 38s 231ms/step - accuracy: 0.7456 - loss: 0.6576 - val_accuracy: 0.7479 - val_loss: 0.6975
Epoch 5/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 38s 231ms/step - accuracy: 0.7744 - loss: 0.5770 - val_accuracy: 0.7583 - val_loss: 0.6498
Epoch 6/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 38s 231ms/step - accuracy: 0.7955 - loss: 0.5409 - val_accuracy: 0.7624 - val_loss: 0.6679
Epoch 7/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 38s 231ms/step - accuracy: 0.8238 - loss: 0.4601 - val_accuracy: 0.7645 - val_loss: 0.6533
Epoch 8/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 38s 231ms/step - accuracy: 0.8289 - loss: 0.4547 - val